##### Copyright 2019 Авторы TensorFlow.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Смешанная точность

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/mixed_precision"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Посмотреть на TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/mixed_precision.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Запустить в Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/mixed_precision.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Посмотреть источник на GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/mixed_precision.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Скачать блокнот</a></td>
</table>

## обзор

Смешанная точность - это использование как 16-битных, так и 32-битных типов с плавающей точкой в модели во время обучения, чтобы она работала быстрее и использовала меньше памяти. Сохраняя определенные части модели в 32-битных типах для числовой стабильности, модель будет иметь меньшее время шага и обучаться одинаково с точки зрения метрик оценки, таких как точность. В этом руководстве описывается, как использовать экспериментальный API Keras со смешанной точностью для ускорения работы ваших моделей. Использование этого API может повысить производительность более чем в 3 раза на современных графических процессорах и на 60% на TPU.

Примечание. API смешанной точности Keras в настоящее время является экспериментальным и может измениться.

Сегодня большинство моделей используют тип float32, который занимает 32 бита памяти. Однако есть два dty-типа с низкой точностью, float16 и bfloat16, каждый из которых занимает 16 бит памяти. Современные ускорители могут выполнять операции быстрее в 16-битных dtypes, поскольку у них есть специальное оборудование для выполнения 16-битных вычислений, а 16-битные dtypes могут считываться из памяти быстрее.

Графические процессоры NVIDIA могут выполнять операции в float16 быстрее, чем в float32, а TPU могут выполнять операции в bfloat16 быстрее, чем float32. Следовательно, эти dty-типы с более низкой точностью следует по возможности использовать на этих устройствах. Тем не менее, переменные и некоторые вычисления все еще должны быть в float32 по численным причинам, чтобы модель работала с тем же качеством. API со смешанной точностью Keras позволяет использовать сочетание float16 или bfloat16 с float32, чтобы получить преимущества производительности от float16 / bfloat16 и преимущества числовой стабильности от float32.

Примечание. В данном руководстве термин «числовая стабильность» относится к тому, как на качество модели влияет использование d-типа с более низкой точностью вместо d-типа с более высокой точностью. Мы говорим, что операция является «численно нестабильной» в float16 или bfloat16, если запуск ее в одном из этих dtypes приводит к худшей точности оценки модели или другим показателям по сравнению с выполнением операции в float32.

## Настроить

API смешанной точности Keras доступен в TensorFlow 2.1.

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.mixed_precision import experimental as mixed_precision

## Поддерживаемое оборудование

Хотя смешанная точность будет работать на большинстве аппаратных средств, она будет ускорять модели только на последних графических процессорах NVIDIA и облачных TPU. Графические процессоры NVIDIA поддерживают использование комбинаций float16 и float32, в то время как TPU поддерживают комбинацию bfloat16 и float32.

Among NVIDIA GPUs, those with compute capability 7.0 or higher will see the greatest performance benefit from mixed precision because they have special hardware units, called Tensor Cores, to accelerate float16 matrix multiplications and convolutions. Older GPUs offer no math performance benefit for using mixed precision, however memory and bandwidth savings can enable some speedups. You can look up the compute capability for your GPU at NVIDIA's [CUDA GPU web page](https://developer.nvidia.com/cuda-gpus). Examples of GPUs that will benefit most from mixed precision include RTX GPUs, the Titan V, and the V100.

Примечание. При запуске этого руководства в Google Colab во время выполнения графического процессора обычно подключается P100. P100 имеет вычислительные возможности 6.0 и не ожидается значительного ускорения.

Вы можете проверить свой тип графического процессора с помощью следующего. Команда существует, только если установлены драйверы NVIDIA, поэтому в противном случае возникнет следующая ошибка.

In [ ]:
!nvidia-smi -L

Все облачные TPU поддерживают bfloat16.

Даже на процессорах и более старых графических процессорах, где ускорение не ожидается, API смешанной точности все еще можно использовать для модульного тестирования, отладки или просто для тестирования API.

## Установка политики dtype

To use mixed precision in Keras, you need to create a `tf.keras.mixed_precision.experimental.Policy`, typically referred to as a *dtype policy*. Dtype policies specify the dtypes layers will run in. In this guide, you will construct a policy from the string `'mixed_float16'` and set it as the global policy. This will will cause subsequently created layers to use mixed precision with a mix of float16 and float32.

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

Политика определяет два важных аспекта уровня: dtype, в котором выполняются вычисления уровня, и dtype переменных уровня. Выше вы создали политику `mixed_float16` (то есть, `mixed_precision.Policy` созданную передачей строки `'mixed_float16'` в ее конструктор). С этой политикой слои используют вычисления float16 и переменные float32. Вычисления выполняются в float16 для повышения производительности, но переменные должны храниться в float32 для числовой стабильности. Вы можете напрямую запросить эти свойства политики.

In [ ]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

As mentioned before, the `mixed_float16` policy will most significantly improve performance on NVIDIA GPUs with compute capability of at least 7.0. The policy will run on other GPUs and CPUs but may not improve performance. For TPUs, the `mixed_bfloat16` policy should be used instead.

## Построение модели

Далее начнем строить простую модель. Очень маленькие модели игрушек, как правило, не выигрывают от смешанной точности, потому что накладные расходы времени выполнения TensorFlow обычно преобладают во времени выполнения, делая любое улучшение производительности на GPU незначительным. Поэтому давайте построим два больших `Dense` слоя по 4096 единиц каждый, если используется графический процессор.

In [ ]:
inputs = keras.Input(shape=(784,), name='digits')
if tf.config.list_physical_devices('GPU'):
  print('The model will run with 4096 units on a GPU')
  num_units = 4096
else:
  # Use fewer units on CPUs so the model finishes in a reasonable amount of time
  print('The model will run with 64 units on a CPU')
  num_units = 64
dense1 = layers.Dense(num_units, activation='relu', name='dense_1')
x = dense1(inputs)
dense2 = layers.Dense(num_units, activation='relu', name='dense_2')
x = dense2(x)

Каждый слой имеет политику и использует глобальную политику по умолчанию. Поэтому каждый из `Dense` слоев имеет политику `mixed_float16` поскольку ранее вы установили глобальную политику на `mixed_float16` . Это заставит плотные слои выполнять вычисления float16 и иметь переменные float32. Они приводят свои входные данные к float16 для выполнения вычислений float16, что приводит к тому, что их выходные данные равны float16. Их переменные имеют тип float32 и будут преобразованы в float16 при вызове слоев, чтобы избежать ошибок из-за несоответствия dtype.

In [ ]:
print('x.dtype: %s' % x.dtype.name)
# 'kernel' is dense1's variable
print('dense1.kernel.dtype: %s' % dense1.kernel.dtype.name)

Затем создайте выходные прогнозы. Обычно вы можете создать выходные прогнозы следующим образом, но это не всегда численно стабильно с float16.

In [ ]:
# INCORRECT: softmax and model output will be float16, when it should be float32
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)

Активация softmax в конце модели должна быть float32. Поскольку политика dtype - `mixed_float16` , активация softmax обычно имеет тип float16, вычисляющий dtype и выводящий тензоры float16.

Это можно исправить, разделив слои Dense и softmax и передав `dtype='float32'` слою softmax.

In [ ]:
# CORRECT: softmax and model output are float32
x = layers.Dense(10, name='dense_logits')(x)
outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)

Передача `dtype='float32'` в конструктор слоя softmax переопределяет политику dtype слоя на политику `float32` , которая выполняет вычисления и сохраняет переменные в float32. Эквивалентно, мы могли бы вместо этого передать `dtype=mixed_precision.Policy('float32')` ; слои всегда преобразуют аргумент dtype в политику. Поскольку слой `Activation` не имеет переменных, переменная политики dtype игнорируется, но вычисление dtype политики float32 приводит к тому, что softmax и выходные данные модели равны float32.

Добавление softmax float16 в середине модели - это хорошо, но softmax в конце модели должен быть в float32. Причина в том, что если промежуточный тензор, текущий от softmax к потере, равен float16 или bfloat16, могут возникнуть числовые проблемы.

Вы можете переопределить dtype любого слоя на float32, передав `dtype='float32'` если считаете, что он не будет численно устойчивым при вычислениях float16. Но, как правило, это необходимо только на последнем слое модели, поскольку большинство слоев имеют достаточную точность с `mixed_float16` и `mixed_bfloat16` .

Даже если модель не заканчивается на softmax, выходные данные все равно должны быть float32. Хотя это и не нужно для этой конкретной модели, выходные данные модели могут быть преобразованы в float32 со следующим:

In [ ]:
# The linear activation is an identity function. So this simply casts 'outputs'
# to float32. In this particular case, 'outputs' is already float32 so this is a
# no-op.
outputs = layers.Activation('linear', dtype='float32')(outputs)

Затем завершите и скомпилируйте модель и сгенерируйте входные данные.

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

В этом примере приводятся входные данные из int8 в float32. Мы не приводим к float16, так как деление на 255 происходит на CPU, который выполняет операции float16 медленнее, чем операции float32. В этом случае разница в производительности незначительна, но в целом вы должны запустить математическую обработку ввода в float32, если он работает на ЦП. Первый слой модели будет преобразовывать входные данные в float16, поскольку каждый слой преобразует входные данные с плавающей запятой в свой тип вычисления d.

Начальные веса модели получены. Это позволит снова тренироваться с нуля, загружая веса.

In [ ]:
initial_weights = model.get_weights()

## Обучение модели с Model.fit

Далее тренируем модель.

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=8192,
                    epochs=5,
                    validation_split=0.2)
test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])


Обратите внимание, что модель печатает время на выборку в журналах: например, «4us / sample». Первая эпоха может быть медленнее, поскольку TensorFlow тратит некоторое время на оптимизацию модели, но после этого время на выборку должно стабилизироваться.

Если вы используете это руководство в Colab, вы можете сравнить производительность смешанной точности с float32. Для этого измените политику с `mixed_float16` на `float32` в разделе «Настройка политики dtype», затем перезапустите все ячейки до этой точки. На графических процессорах с по крайней мере 7.0 вычислительными возможностями вы должны значительно увеличить время на выборку, что указывает на смешанную точность, ускоряющую модель. Например, для графического процессора Titan V время выборки увеличивается с 4 до 12 мкс. Обязательно измените политику обратно на `mixed_float16` и перезапустите ячейки, прежде чем продолжить с руководством.

Для многих реальных моделей смешанная точность также позволяет удвоить размер пакета без исчерпания памяти, поскольку тензоры float16 занимают половину памяти. Однако это не относится к этой игрушечной модели, так как вы, вероятно, можете запустить модель в любом типе d, где каждый пакет состоит из всего набора данных MNIST из 60000 изображений.

If running mixed precision on a TPU, you will not see as much of a performance gain compared to running mixed precision on GPUs. This is because TPUs already do certain ops in bfloat16 under the hood even with the default dtype policy of `float32`. TPU hardware does not support float32 for certain ops which are numerically stable in bfloat16, such as matmul. For such ops the TPU backend will silently use bfloat16 internally instead. As a consequence, passing `dtype='float32'` to layers which use such ops may have no numerical effect, however it is unlikely running such layers with bfloat16 computations will be harmful.

## Масштабирование потерь

Loss scaling is a technique which `tf.keras.Model.fit` automatically performs with the `mixed_float16` policy to avoid numeric underflow. This section describes loss scaling and how to customize its behavior.

### Недостаток и переполнение

Тип данных float16 имеет узкий динамический диапазон по сравнению с float32. Это означает, что значения выше $ 65504 $ будут переполнены до бесконечности, а значения ниже $ 6.0 \ times 10 ^ {- 8} $ опустятся до нуля. У float32 и bfloat16 гораздо больший динамический диапазон, поэтому переполнение и потеря значения не являются проблемой.

Например:

In [ ]:
x = tf.constant(256, dtype='float16')
(x ** 2).numpy()  # Overflow

In [ ]:
x = tf.constant(1e-5, dtype='float16')
(x ** 2).numpy()  # Underflow

На практике переполнение с помощью float16 происходит редко. Кроме того, во время прямого прохода также наблюдается недостаточный расход. Однако во время обратного прохода градиенты могут опуститься до нуля. Масштабирование потерь - это методика предотвращения этого недостатка.

### Потеря масштабирования фона

Основная концепция масштабирования убытков проста: просто умножьте потери на какое-то большое число, скажем, $ 1024 $. Мы называем этот номер *шкалой потерь* . Это также приведет к масштабированию градиентов на 1024 долл. США, что значительно снизит вероятность недостаточного использования. Как только окончательные градиенты вычислены, разделите их на $ 1024 $, чтобы вернуть их к верным значениям.

Псевдокод этого процесса:

```
loss_scale = 1024 loss = model(inputs) loss *= loss_scale # We assume `grads` are float32. We do not want to divide float16 gradients grads = compute_gradient(loss, model.trainable_variables) grads /= loss_scale
```

Выбор масштаба потерь может быть сложным. Если масштаб потерь слишком низок, градиенты могут все еще опуститься до нуля. Если оно слишком высокое, возникает противоположная проблема: градиенты могут перетекать в бесконечность.

Чтобы решить эту проблему, TensorFlow динамически определяет масштаб потерь, поэтому вам не нужно выбирать один вручную. Если вы используете `tf.keras.Model.fit` , масштабирование убытков выполнено за вас, поэтому вам не нужно делать никакой дополнительной работы. Это объясняется далее в следующем разделе.


### Выбор масштаба потерь

Каждая политика dtype дополнительно имеет связанный объект `tf.mixed_precision.experimental.LossScale` , который представляет фиксированную или динамическую шкалу потерь. По умолчанию шкалой потерь для политики `mixed_float16` является `tf.mixed_precision.experimental.DynamicLossScale` , который динамически определяет значение шкалы потерь. Другие политики по умолчанию не имеют шкалы потерь, так как это необходимо только при использовании float16. Вы можете запросить масштаб потери политики:

In [ ]:
loss_scale = policy.loss_scale
print('Loss scale: %s' % loss_scale)

Шкала потерь печатает много внутреннего состояния, но вы можете игнорировать его. Наиболее важной частью является часть `current_loss_scale` , которая показывает текущее значение масштаба потерь.

Вместо этого вы можете использовать статическую шкалу потерь, передавая число при построении политики dtype.

In [ ]:
new_policy = mixed_precision.Policy('mixed_float16', loss_scale=1024)
print(new_policy.loss_scale)

Конструктор политики dtype всегда преобразует масштаб потерь в объект `LossScale` . В этом случае он преобразуется в `tf.mixed_precision.experimental.FixedLossScale` , единственный другой подкласс `LossScale` отличный от `DynamicLossScale` .

Примечание. *Использование чего-либо, кроме динамической шкалы потерь, не рекомендуется* . Выбор фиксированной шкалы потерь может быть затруднен, так как ее слишком низкое значение приведет к тому, что модель также не будет обучаться, а слишком высокое - к появлению Infs или NaN в градиентах. Динамическая шкала потерь обычно близка к оптимальной шкале потерь, поэтому вам не нужно выполнять какую-либо работу. В настоящее время шкалы динамических потерь немного медленнее шкал с фиксированными потерями, но в будущем производительность будет улучшена.

Модели, как и слои, имеют политику dtype. При наличии модель использует масштаб потерь своей политики для применения масштабирования потерь в методе `tf.keras.Model.fit` . Это означает, что если используется `Model.fit` , вам вообще не нужно беспокоиться о масштабировании потерь: политика `mixed_float16` по умолчанию будет иметь динамический масштаб потерь, и `Model.fit` будет применять его.

При использовании пользовательских циклов обучения модель будет игнорировать шкалу потерь политики, и вам придется применять ее вручную. Это объясняется в следующем разделе.

## Тренировка модели с индивидуальной тренировочной петлей

So far, you trained a Keras model with mixed precision using `tf.keras.Model.fit`. Next, you will use mixed precision with a custom training loop. If you do not already know what a custom training loop is, please read [the Custom training guide](../tutorials/customization/custom_training_walkthrough.ipynb) first.

Выполнение пользовательского цикла обучения со смешанной точностью требует двух изменений по сравнению с его запуском в float32:

1. Постройте модель со смешанной точностью (вы уже сделали это)
2. Явно используйте масштабирование потерь, если используется `mixed_float16` .


For step (2), you will use the `tf.keras.mixed_precision.experimental.LossScaleOptimizer` class, which wraps an optimizer and applies loss scaling. It takes two arguments: the optimizer and the loss scale. Construct one as follows to use a dynamic loss scale

In [ ]:
optimizer = keras.optimizers.RMSprop()
optimizer = mixed_precision.LossScaleOptimizer(optimizer, loss_scale='dynamic')

Передача `'dynamic'` эквивалентна передаче `tf.mixed_precision.experimental.DynamicLossScale()` .

Затем определите объект потерь и `tf.data.Dataset` .

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
train_dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_train))
                 .shuffle(10000).batch(8192))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(8192)

Затем определите функцию шага тренировки. Два новых метода из оптимизатора масштаба потерь используются для масштабирования потерь и масштабирования градиентов:

- `get_scaled_loss(loss)`: Multiplies the loss by the loss scale
- `get_unscaled_gradients(gradients)` : принимает список масштабированных градиентов в качестве входных данных и делит каждый из них на шкалу потерь для их масштабирования

Эти функции должны использоваться для предотвращения переполнения градиентов. `LossScaleOptimizer.apply_gradients` будет затем применять градиенты, если ни один из них не имеет Infs или NaN. Это также обновит масштаб потерь, сократив его вдвое, если градиенты имели Infs или NaN, и потенциально увеличивая его в противном случае.

In [ ]:
@tf.function
def train_step(x, y):
  with tf.GradientTape() as tape:
    predictions = model(x)
    loss = loss_object(y, predictions)
    scaled_loss = optimizer.get_scaled_loss(loss)
  scaled_gradients = tape.gradient(scaled_loss, model.trainable_variables)
  gradients = optimizer.get_unscaled_gradients(scaled_gradients)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

`LossScaleOptimizer` , вероятно, пропустит первые несколько шагов в начале обучения. Шкала потерь начинается высоко, поэтому можно быстро определить оптимальный масштаб потерь. После нескольких шагов шкала потерь стабилизируется и будет пропущено очень мало шагов. Этот процесс происходит автоматически и не влияет на качество обучения.

Теперь определите шаг теста.


In [ ]:
@tf.function
def test_step(x):
  return model(x, training=False)

Загрузите начальный вес модели, чтобы вы могли переучивать с нуля.

In [ ]:
model.set_weights(initial_weights)

Наконец, запустите пользовательский цикл обучения.

In [ ]:
for epoch in range(5):
  epoch_loss_avg = tf.keras.metrics.Mean()
  test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='test_accuracy')
  for x, y in train_dataset:
    loss = train_step(x, y)
    epoch_loss_avg(loss)
  for x, y in test_dataset:
    predictions = test_step(x)
    test_accuracy.update_state(y, predictions)
  print('Epoch {}: loss={}, test accuracy={}'.format(epoch, epoch_loss_avg.result(), test_accuracy.result()))

## Советы по производительности GPU

Вот несколько советов по производительности при использовании смешанной точности на графических процессорах.

### Увеличение размера партии

Если это не влияет на качество модели, попробуйте запустить с двойным размером партии при использовании смешанной точности. Так как тензоры float16 используют половину памяти, это часто позволяет удвоить размер пакета без исчерпания памяти. Увеличение размера пакета, как правило, увеличивает пропускную способность обучения, т.е. количество элементов в секунду, на которых может работать ваша модель.

### Обеспечение использования тензорных ядер GPU

Как упоминалось ранее, современные графические процессоры NVIDIA используют специальное аппаратное устройство, называемое Tensor Core, которое может очень быстро размножать матрицы с плавающей точкой16. Однако для тензорных ядер требуются определенные размеры тензоров, кратные 8. В приведенных ниже примерах аргумент выделяется жирным шрифтом тогда и только тогда, когда он должен быть кратным 8 для использования тензорных ядер.

- tf.keras.layers.Dense ( **единиц = 64** )
- tf.keras.layers.Conv2d ( **filters = 48** , kernel_size = 7, stride = 3)
    - И аналогично для других сверточных слоев, таких как tf.keras.layers.Conv3d
- tf.keras.layers.LSTM ( **единиц = 64** )
    - И аналогично для других RNN, таких как tf.keras.layers.GRU
- tf.keras.Model.fit (epochs = 2, **batch_size = 128** )

Вы должны попытаться использовать Тензорные Ядра, когда это возможно. Если вы хотите узнать больше, [руководство по производительности глубокого обучения NVIDIA](https://docs.nvidia.com/deeplearning/sdk/dl-performance-guide/index.html) описывает точные требования к использованию ядер Tensor, а также другую информацию о производительности, связанную с Tensor Core.

### XLA

XLA - это компилятор, который может еще больше повысить производительность смешанной точности, а также производительность float32 в меньшей степени. См. [Руководство XLA](https://www.tensorflow.org/xla) для деталей.

## Советы по производительности облачного TPU

Как и на графических процессорах, вы должны попробовать удвоить размер пакета, так как тензоры bfloat16 используют половину памяти. Удвоение размера партии может увеличить пропускную способность обучения.

TPUs do not require any other mixed precision-specific tuning to get optimal performance. TPUs already require the use of XLA. They benefit from having certain dimensions being multiples of $128$, but this applies equally to float32 as it does for mixed precision. See the [Cloud TPU Performance Guide](https://cloud.google.com/tpu/docs/performance-guide) for general TPU performance tips, which apply to mixed precision as well as float32.

## Резюме

- Вам следует использовать смешанную точность, если вы используете TPU или графические процессоры NVIDIA с как минимум вычислительной способностью 7.0, поскольку это повысит производительность в 3 раза.
- Вы можете использовать смешанную точность со следующими строками:
    ```
    # On TPUs, use 'mixed_bfloat16' instead policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16') mixed_precision.set_policy(policy)
    ```
- Если ваша модель заканчивается на softmax, убедитесь, что это float32. И независимо от того, чем заканчивается ваша модель, убедитесь, что вывод float32.
- Если вы используете пользовательский обучающий цикл с `mixed_float16` , в дополнение к вышеприведенным строкам вам нужно обернуть свой оптимизатор в `tf.keras.mixed_precision.experimental.LossScaleOptimizer` . Затем вызовите `optimizer.get_scaled_loss` для масштабирования потерь и `optimizer.get_unscaled_gradients` для масштабирования градиентов.
- Удвойте размер тренировочной партии, если это не снижает точность оценки
- На GPU убедитесь, что большинство тензорных измерений кратно 8 $, чтобы максимизировать производительность

Дополнительные примеры смешанной точности с использованием API `tf.keras.mixed_precision` см. В [официальном репозитории моделей](https://github.com/tensorflow/models/tree/master/official) . Большинство официальных моделей, таких как [ResNet](https://github.com/tensorflow/models/tree/master/official/vision/image_classification) и [Transformer,](https://github.com/tensorflow/models/blob/master/official/nlp/transformer) будут работать с смешанной точностью, передавая `--dtype=fp16` .
